In [18]:
import pandas as pd
import numpy as np
from dash_website import DIMENSIONS, MAIN_CATEGORIES_TO_CATEGORIES

list_indexes = []
for dimension_1 in DIMENSIONS:
    for dimension_2 in DIMENSIONS:
        for category in MAIN_CATEGORIES_TO_CATEGORIES["All"] + ["Phenotypic", "Genetics"] + [f"All_{main_category}" for main_category in MAIN_CATEGORIES_TO_CATEGORIES.keys()]:
            list_indexes.append([dimension_1, dimension_2, category])
indexes = pd.MultiIndex.from_tuples(list_indexes, names=["dimension_1", "dimension_2", "category"])

list_columns = []
for method in ["all", "union", "intersection"]:
    for correlation_type in ["pearson", "spearman", "number_variables"]:
        list_columns.append([method, correlation_type])
columns = pd.MultiIndex.from_tuples(list_columns, names=["method", "correlation_type"])
correlations = pd.DataFrame(None, index=indexes, columns=columns)

phenotypic = pd.read_feather("../../data/xwas/univariate_correlations/phenotypic.feather").set_index(["dimension_1", "dimension_2"])
genetics = pd.read_feather("../../data/xwas/univariate_correlations/genetics.feather").set_index(["dimension_1", "dimension_2"])

In [31]:
dimension_1, dimension_2 = "Abdomen", "EyesAll"

In [32]:
def load_correlation(key_in_aws):
    correlation_dimension = pd.read_feather(key_in_aws)
    correlation_dimension.drop(index=correlation_dimension.index[correlation_dimension["sample_size"] < 10], inplace=True)

    return correlation_dimension.set_index(["category", "variable"])

In [33]:
correlation_dimension_1 = load_correlation(f"../../data/xwas/univariate_results/linear_correlations_{dimension_1}.feather")
correlation_dimension_2 = load_correlation(f"../../data/xwas/univariate_results/linear_correlations_{dimension_2}.feather")

In [34]:
for category in MAIN_CATEGORIES_TO_CATEGORIES["All"] + [f"All_{main_category}" for main_category in MAIN_CATEGORIES_TO_CATEGORIES.keys()]:
    if "All" in category:
        categories = MAIN_CATEGORIES_TO_CATEGORIES[category.split("_")[1]]
    else:
        categories = [category]

    correlations_1 = correlation_dimension_1.loc[categories]
    correlations_2 = correlation_dimension_2.loc[categories]

    indexes = {}

    indexes["all"] = correlations_1.index.intersection(correlations_2.index)

    indexes["union"] = correlations_1.index[
        correlations_1["p_value"] < 0.05 / correlations_1.shape[0]
    ].union(correlations_2.index[correlations_2["p_value"] < 0.05 / correlations_2.shape[0]])
    if (~indexes["union"].isin(correlations_1.index)).sum() > 0:
        correlations_to_add = pd.DataFrame(None, index=indexes["union"][~indexes["union"].isin(correlations_1.index)], columns=["p_value", "correlation", "sample_size"])
        correlations_to_add["correlation"] = 0

        correlations_1 = correlations_1.append(correlations_to_add)
    if (~indexes["union"].isin(correlations_2.index)).sum() > 0:
        correlations_to_add = pd.DataFrame(None, index=indexes["union"][~indexes["union"].isin(correlations_2.index)], columns=["p_value", "correlation", "sample_size"])
        correlations_to_add["correlation"] = 0

        correlations_2 = correlations_2.append(correlations_to_add)

    indexes["intersection"] = correlations_1.index[correlations_1["p_value"] < 0.05 / correlations_1.shape[0]].intersection(correlations_2.index[correlations_2["p_value"] < 0.05 / correlations_2.shape[0]])

    for method in ["all", "union", "intersection"]:
        for correlation_type in ["pearson", "spearman"]:
            if len(indexes[method]) <= 1:
                correlations.loc[(dimension_1, dimension_2, category), (method, correlation_type)] = 0
            else:
                correlations.loc[(dimension_1, dimension_2, category), (method, correlation_type)] = correlations_1.loc[indexes[method], "correlation"].corr(correlations_2.loc[indexes[method], "correlation"], method=correlation_type)
            correlations.loc[(dimension_1, dimension_2, category), (method, "number_variables")] = len(indexes[method])

for method in ["all", "union", "intersection"]:
    for correlation_type in ["pearson", "spearman"]:
        correlations.loc[(dimension_1, dimension_2, "Phenotypic"), (method, correlation_type)] = phenotypic.loc[(dimension_1, dimension_2), "correlation"]
        correlations.loc[(dimension_1, dimension_2, "Genetics"), (method, correlation_type)] = genetics.loc[(dimension_1, dimension_2), "correlation"]

c:\Dossier\HMS\Dash-Website\env_website\lib\site-packages\numpy\lib\function_base.py:2634: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Dossier\HMS\Dash-Website\env_website\lib\site-packages\numpy\lib\function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Dossier\HMS\Dash-Website\env_website\lib\site-packages\numpy\lib\function_base.py:2634: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Dossier\HMS\Dash-Website\env_website\lib\site-packages\numpy\lib\function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


KeyError: ('Abdomen', 'EyesAll')

In [30]:
correlations.loc[(dimension_1, dimension_2)]

method                       all                                union  \
correlation_type         pearson  spearman number_variables   pearson   
category                                                                
Alcohol                 0.509463  0.286207               29  0.371599   
AnthropometryBodySize  -0.465823 -0.333333                8 -0.465823   
AnthropometryImpedance  0.998125       1.0                5  0.998125   
ArterialStiffness       0.923676  0.833333                8 -0.326431   
BloodBiochemistry       0.477404  0.548987               28  0.476419   
...                          ...       ...              ...       ...   
All_Biomarkers          0.331183  0.368207             3115   0.37547   
All_Phenotypes           0.11673   0.37918              182  0.689089   
All_Diseases            0.501592  0.397664             2073   0.50459   
All_Environmental       0.420648  0.543085              263  0.664054   
All_Socioeconomics       0.15739  0.400414               85  0.741935   

method                                            intersection            \
correlation_type        spearman number_variables      pearson  spearman   
category                                                                   
Alcohol                 0.181818               12    -0.033912 -0.535714   
AnthropometryBodySize  -0.333333                8    -0.651772 -0.771429   
AnthropometryImpedance       1.0                5     0.997697       1.0   
ArterialStiffness           -0.4                4          1.0       1.0   
BloodBiochemistry       0.551893               27     0.576583  0.516484   
...                          ...              ...          ...       ...   
All_Biomarkers          0.368507              648     0.329282  0.314841   
All_Phenotypes           0.57594               56     0.734219  0.313846   
All_Diseases            0.301089              223     0.778171  0.619481   
All_Environmental        0.62875              108     0.741077  0.703831   
All_Socioeconomics      0.710837               29     0.760276  0.582418   

method                                   
correlation_type       number_variables  
category                                 
Alcohol                               7  
AnthropometryBodySize                 6  
AnthropometryImpedance                4  
ArterialStiffness                     2  
BloodBiochemistry                    13  
...                                 ...  
All_Biomarkers                       37  
All_Phenotypes                       25  
All_Diseases                         21  
All_Environmental                    54  
All_Socioeconomics                   13  

[92 rows x 9 columns]

In [29]:
correlations.columns = map(str, correlations.columns.tolist())
d = correlations.reset_index()
d

,dimension_1,dimension_2,category,"('all', 'pearson')","('all', 'spearman')","('all', 'number_variables')","('union', 'pearson')","('union', 'spearman')","('union', 'number_variables')","('intersection', 'pearson')","('intersection', 'spearman')","('intersection', 'number_variables')"
0,Abdomen,Abdomen,Alcohol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abdomen,Abdomen,AnthropometryBodySize,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abdomen,Abdomen,AnthropometryImpedance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Abdomen,Abdomen,ArterialStiffness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abdomen,Abdomen,BloodBiochemistry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
100183,set_instances23,set_instances23,All_Biomarkers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100184,set_instances23,set_instances23,All_Phenotypes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100185,set_instances23,set_instances23,All_Diseases,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100186,set_instances23,set_instances23,All_Environmental,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
correlations = d.set_index(["dimension_1", "dimension_2", "category"])
correlations.columns = pd.MultiIndex.from_tuples(
    list(map(eval, correlations.columns.tolist())), names=["category", "variable"]
)
correlations

category                                                   all           \
variable                                               pearson spearman   
dimension_1     dimension_2     category                                  
Abdomen         Abdomen         Alcohol                    NaN      NaN   
                                AnthropometryBodySize      NaN      NaN   
                                AnthropometryImpedance     NaN      NaN   
                                ArterialStiffness          NaN      NaN   
                                BloodBiochemistry          NaN      NaN   
...                                                        ...      ...   
set_instances23 set_instances23 All_Biomarkers             NaN      NaN   
                                All_Phenotypes             NaN      NaN   
                                All_Diseases               NaN      NaN   
                                All_Environmental          NaN      NaN   
                                All_Socioeconomics         NaN      NaN   

category                                                                 \
variable                                               number_variables   
dimension_1     dimension_2     category                                  
Abdomen         Abdomen         Alcohol                             NaN   
                                AnthropometryBodySize               NaN   
                                AnthropometryImpedance              NaN   
                                ArterialStiffness                   NaN   
                                BloodBiochemistry                   NaN   
...                                                                 ...   
set_instances23 set_instances23 All_Biomarkers                      NaN   
                                All_Phenotypes                      NaN   
                                All_Diseases                        NaN   
                                All_Environmental                   NaN   
                                All_Socioeconomics                  NaN   

category                                                 union           \
variable                                               pearson spearman   
dimension_1     dimension_2     category                                  
Abdomen         Abdomen         Alcohol                    NaN      NaN   
                                AnthropometryBodySize      NaN      NaN   
                                AnthropometryImpedance     NaN      NaN   
                                ArterialStiffness          NaN      NaN   
                                BloodBiochemistry          NaN      NaN   
...                                                        ...      ...   
set_instances23 set_instances23 All_Biomarkers             NaN      NaN   
                                All_Phenotypes             NaN      NaN   
                                All_Diseases               NaN      NaN   
                                All_Environmental          NaN      NaN   
                                All_Socioeconomics         NaN      NaN   

category                                                                 \
variable                                               number_variables   
dimension_1     dimension_2     category                                  
Abdomen         Abdomen         Alcohol                             NaN   
                                AnthropometryBodySize               NaN   
                                AnthropometryImpedance              NaN   
                                ArterialStiffness                   NaN   
                                BloodBiochemistry                   NaN   
...                                                                 ...   
set_instances23 set_instances23 All_Biomarkers                      NaN   
                                All_Phenotypes                      NaN   
                      

In [46]:
sorted(phenotypic.index.get_level_values(0).drop_duplicates())

['Abdomen',
 'AbdomenLiver',
 'AbdomenPancreas',
 'Arterial',
 'ArterialCarotids',
 'ArterialPulseWaveAnalysis',
 'Biochemistry',
 'BiochemistryBlood',
 'BiochemistryUrine',
 'Brain',
 'BrainCognitive',
 'BrainMRI',
 'Eyes',
 'EyesAll',
 'EyesFundus',
 'EyesOCT',
 'Hearing',
 'Heart',
 'HeartECG',
 'HeartMRI',
 'ImmuneSystem',
 'Lungs',
 'Musculoskeletal',
 'MusculoskeletalFullBody',
 'MusculoskeletalHips',
 'MusculoskeletalKnees',
 'MusculoskeletalScalars',
 'MusculoskeletalSpine',
 'PhysicalActivity',
 'set',
 'set_instances01',
 'set_instances1.5x',
 'set_instances23']